In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [3]:
def data_concat(df,path):
    for p in path:
        data = pd.read_csv(p,header=None)
        df = pd.concat([df,data],axis=0)
    return df

In [4]:
# 取PSD特征
def psd_feature(df):
    m=df.shape[1]
    n=100   #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        df_n = 10**(df_n/10)
        mean=df_n.mean(axis=1)
        var = df_n.var(axis=1)
        mean=mean.map(lambda x :math.log(x,10)*10)
        signal_feature['psd_mean_'+str(i)] = mean
        signal_feature['psd_var_'+str(i)] = var
    return signal_feature

In [5]:
# 取PSD差分特征
def psd_cha_feature(df):
    m=df.shape[1]
    n=100
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        mean=df_n.mean(axis=1)    # mean 代表该频段的平均上升下降次数
        #mean=mean.map(lambda x:0 if x>-0.5 else 1) # 二次离散化
        signal_feature['psd_cha_'+str(i)] = mean
    return signal_feature

In [6]:
def convert_cha(sa):
    if (sa>0):
        return 1
    else:
        return -1

In [7]:
# 取差值（斜率）作为特征
def cha(df):
    n=df.shape[1]
    ret = pd.DataFrame()
    for i in range(n-1):
        d=df.iloc[:,i+1]-df.iloc[:,i]
        ret[i]=d
    ret[n-1]=0
    return ret

In [8]:
path_all=['./matlab/test_data_UAV_f_20.csv','./matlab/test_data_UAV_f_10.csv','./matlab/test_data_UAV_f_0.csv','./matlab/test_data_UAV_f_-10.csv','./matlab/test_data_UAV_f_-12.csv',\
         './matlab/test_data_UAV_f_-15.csv','./matlab/test_data_UAV_f_-18.csv','./matlab/test_data_UAV_f_-20.csv','./matlab/test_data_UAV_f_-30.csv','./matlab/test_data_noise_f.csv']

In [9]:
all_data = pd.DataFrame()
all_data = data_concat(all_data,path_all)
all_data.shape

(1400, 5000)

In [10]:
all_data=all_data.iloc[:,range(1500,4500)]
all_data.shape

(1400, 3000)

In [11]:
all_data_cha = cha(all_data)
all_data_cha.shape

(1400, 3000)

In [12]:
all_data.index=range(all_data.shape[0])
all_data_cha.index=range(all_data_cha.shape[0])

In [13]:
# train_feature 为最终的训练特征
all_feature = pd.DataFrame()

In [14]:
psd = psd_feature(all_data)
psd = psd.sort_index(axis=1)
#psd_mean_feature=psd.iloc[:,range(30)].applymap(convert_data)
#psd_var_feature=psd.iloc[:,range(30,60)].applymap(convert_var)

In [15]:
all_data_cha_cate = all_data_cha.applymap(convert_cha)
psd_cha = psd_cha_feature(all_data_cha_cate)
psd_cha.shape

(1400, 30)

In [16]:
all_feature = pd.concat([psd,psd_cha],axis=1)
all_feature.shape

(1400, 90)

In [16]:
all_feature

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
0,-67.282532,-66.898247,-66.969016,-67.007096,-66.932562,-66.966760,-66.969645,-67.012851,-66.938821,-66.997767,...,0.06,-0.04,-0.04,-0.02,-0.26,-0.32,-0.02,0.04,0.06,-0.10
1,-67.241512,-66.951386,-67.007373,-66.943422,-66.988167,-67.033702,-67.026743,-67.006855,-67.039875,-67.040460,...,0.00,-0.10,0.00,-0.02,-0.12,-0.28,-0.02,-0.02,-0.06,-0.10
2,-67.218958,-67.042780,-67.063387,-66.959391,-66.969810,-66.914690,-67.020641,-66.930323,-67.055691,-67.036016,...,-0.04,0.06,-0.04,-0.06,-0.18,-0.26,-0.02,-0.02,-0.02,-0.14
3,-67.248935,-66.921824,-67.050632,-67.022052,-66.958460,-66.974205,-66.934288,-66.987164,-67.032370,-66.945881,...,-0.06,-0.02,0.04,-0.04,-0.30,-0.16,-0.10,0.06,0.00,0.10
4,-67.219442,-66.970364,-67.037770,-67.036419,-66.996777,-66.937282,-66.975397,-66.994155,-67.074694,-66.999406,...,0.06,0.08,0.04,0.00,-0.20,-0.18,-0.06,0.06,-0.06,0.08
5,-67.339371,-66.930350,-67.006520,-67.006655,-66.964616,-67.018293,-66.937713,-67.012804,-67.052241,-67.005228,...,0.02,0.06,-0.02,-0.06,-0.14,-0.10,0.02,-0.10,-0.06,-0.02
6,-67.342473,-66.963737,-66.984756,-67.008215,-66.995281,-66.983374,-67.013807,-66.955271,-66.940273,-66.967238,...,0.04,0.04,-0.04,0.00,-0.12,-0.28,0.00,0.10,0.02,0.10
7,-67.329444,-66.967029,-67.068241,-66.963744,-67.010699,-66.958400,-66.953718,-66.948951,-67.041492,-66.996301,...,-0.06,0.14,0.00,0.00,-0.20,-0.26,0.04,-0.08,0.02,0.06
8,-67.280649,-66.940746,-67.000650,-66.981304,-66.994792,-66.999036,-67.011897,-67.003051,-67.012102,-66.979989,...,-0.02,-0.06,-0.08,-0.04,-0.26,-0.26,0.04,0.02,-0.04,-0.04
9,-67.293630,-66.890342,-67.042277,-67.006970,-67.025457,-66.905209,-66.911519,-66.958070,-67.008140,-66.998522,...,0.08,-0.04,0.00,0.00,-0.22,-0.26,-0.02,0.00,0.12,0.00


In [24]:
all_feature.iloc[range(100,200),:]

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
100,-77.058271,-76.963830,-76.916491,-76.800936,-77.021759,-77.023463,-76.851468,-76.707397,-76.935687,-77.151274,...,-0.08,-0.06,0.08,0.08,-0.08,-0.12,-0.02,0.08,-0.06,-0.02
101,-77.085857,-76.795903,-77.171243,-76.836816,-77.011539,-77.114100,-76.935616,-76.886971,-77.035097,-76.854010,...,-0.04,-0.04,-0.08,0.00,0.04,-0.02,0.04,-0.12,0.08,0.08
102,-76.893428,-76.860112,-77.038966,-76.963662,-76.994416,-77.026592,-76.886252,-76.956142,-76.944080,-76.816562,...,-0.02,0.00,0.12,0.00,-0.04,-0.12,0.02,-0.04,0.02,0.00
103,-77.173295,-77.074685,-76.869307,-77.092329,-76.961146,-76.820347,-77.109340,-77.046269,-76.704154,-76.781116,...,0.00,0.02,-0.08,0.04,-0.08,-0.14,-0.06,-0.12,-0.08,0.00
104,-77.343545,-76.850169,-77.060551,-76.677625,-77.177832,-76.818825,-76.695335,-76.907733,-76.932566,-76.726311,...,0.08,-0.08,0.02,-0.10,0.02,0.00,-0.14,-0.02,0.02,-0.02
105,-77.510476,-76.981740,-76.880376,-76.939046,-77.002243,-77.012176,-77.013875,-77.081918,-76.841585,-77.116677,...,-0.08,0.10,-0.10,-0.02,-0.10,-0.04,0.00,-0.04,0.00,0.02
106,-77.175726,-77.106840,-76.976208,-76.868256,-76.910503,-76.943783,-77.033661,-77.175092,-76.710898,-76.838572,...,-0.04,-0.02,0.04,-0.02,-0.04,-0.06,-0.04,0.04,-0.06,-0.06
107,-77.245079,-76.882533,-77.182171,-77.017332,-76.967631,-77.117268,-77.087423,-76.971636,-76.937173,-76.813740,...,0.02,0.10,-0.12,0.14,-0.06,-0.04,0.06,-0.02,-0.02,-0.06
108,-77.323628,-77.126972,-77.034381,-76.839749,-76.933486,-77.069767,-76.811515,-77.068479,-76.662548,-76.992957,...,0.00,0.08,0.00,0.02,-0.06,-0.02,-0.02,0.04,0.00,-0.04
109,-77.065627,-76.914467,-76.957018,-76.992355,-76.671310,-76.908358,-76.815998,-76.945391,-77.057111,-76.594917,...,0.04,-0.12,0.06,-0.06,-0.20,-0.04,-0.06,0.10,-0.08,0.04


In [26]:
all_feature.loc[100].plot()

In [17]:
data_all=MinMaxScaler().fit_transform(all_feature) #normalization 得到numpy array

In [18]:
all_data = pd.DataFrame(data_all)

In [19]:
all_data.columns=all_feature.columns

In [20]:
all_data['label']=[1]*900+[0]*500
all_data

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29,label
0,0.997492,0.998809,0.995360,0.993796,0.998443,0.995942,0.996128,0.994758,0.998181,0.996014,...,0.35,0.421053,0.50,0.115385,0.074074,0.428571,0.60,0.60,0.30,1
1,0.999027,0.996795,0.993912,0.996188,0.996342,0.993408,0.993998,0.994983,0.994364,0.994411,...,0.20,0.526316,0.50,0.384615,0.148148,0.428571,0.45,0.30,0.30,1
2,0.999871,0.993331,0.991798,0.995588,0.997036,0.997913,0.994226,0.997864,0.993767,0.994578,...,0.60,0.421053,0.40,0.269231,0.185185,0.428571,0.45,0.40,0.20,1
3,0.998749,0.997916,0.992279,0.993235,0.997465,0.995660,0.997446,0.995724,0.994648,0.997961,...,0.40,0.631579,0.45,0.038462,0.370370,0.238095,0.65,0.45,0.80,1
4,0.999853,0.996076,0.992765,0.992695,0.996017,0.997058,0.995913,0.995461,0.993049,0.995952,...,0.65,0.631579,0.55,0.230769,0.333333,0.333333,0.65,0.30,0.75,1
5,0.995365,0.997593,0.993944,0.993813,0.997232,0.993991,0.997319,0.994759,0.993897,0.995734,...,0.60,0.473684,0.40,0.346154,0.481481,0.523810,0.25,0.30,0.50,1
6,0.995249,0.996327,0.994766,0.993754,0.996073,0.995313,0.994481,0.996925,0.998126,0.997160,...,0.55,0.421053,0.55,0.384615,0.148148,0.476190,0.75,0.50,0.80,1
7,0.995736,0.996202,0.991615,0.995424,0.995491,0.996259,0.996722,0.997163,0.994303,0.996069,...,0.80,0.526316,0.55,0.230769,0.185185,0.571429,0.30,0.50,0.70,1
8,0.997562,0.997198,0.994166,0.994765,0.996092,0.994720,0.994552,0.995126,0.995413,0.996681,...,0.30,0.315789,0.45,0.115385,0.185185,0.571429,0.55,0.35,0.45,1
9,0.997077,0.999109,0.992595,0.993801,0.994933,0.998272,0.998295,0.996820,0.995563,0.995985,...,0.35,0.526316,0.55,0.192308,0.185185,0.428571,0.50,0.75,0.55,1


In [21]:
all_data.to_csv('test_data_1221.csv',index=False)